In [1]:
import svfpy
import rasterio
import numpy as np
# import zarr

In [2]:
mds = '../test/data/mds_sao_paulo_city_1000.tiff'

In [3]:
svf = svfpy.SVF(mds_path=mds, kernel_size=11)

In [4]:
profile = svf.mds_src.profile

In [5]:
result_temp = svfpy.calculate(svf)

(0, 6)
(6, 9)
(9, 12)
(12, 15)
(15, 18)
(18, 21)
(21, 24)
(24, 27)
(27, 30)


In [6]:
result_temp[500, 500, :, 1:]

array([[ 1.12988281,  0.81494141,  0.50634766, ..., -0.0508728 ,
         0.03826904,  0.        ],
       [ 1.12988281,  0.93017578,  0.50634766, ...,  0.02816772,
         0.04275513,  0.        ],
       [ 1.24902344,  0.72949219,  0.56396484, ...,  0.03210449,
         0.03491211,  0.        ],
       ...,
       [-1.37109375,  0.72949219,  0.27050781, ..., -0.06274414,
         0.04275513,  0.        ],
       [-1.37109375,  0.61572266,  0.50634766, ..., -0.0508728 ,
         0.03826904,  0.        ],
       [-1.51171875,  0.81494141,  0.44067383, ..., -0.04324341,
         0.03381348,  0.        ]])

In [58]:
seila = np.argmax(result_temp[:500, :500, :, 1:], axis=3)

In [63]:
# result_temp[:500, :500, :]
# np.arctan2(0.0000000001,0.00000000001)

1.4711276743037347

In [8]:
result_temp.info

Type,zarr.core.Array
Data type,float64
Shape,"(1000, 1000, 40, 30)"
Chunk shape,"(125, 125, 5, 8)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.KVStore
No. bytes,9600000000 (8.9G)
No. bytes stored,1792527154 (1.7G)
Storage ratio,5.4


In [13]:
result_temp[:500, :500]

array([[[[ 1.57031250e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -5.08728027e-02,  3.82690430e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           2.81677246e-02,  4.27551270e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           3.21044922e-02,  3.49121094e-02,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -6.27441406e-02,  4.27551270e-02,  0.00000000e+00],
         [ 1.57031250e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -5.08728027e-02,  3.82690430e-02,  0.00000000e+00],
         [ 1.57031250e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -4.32434082e-02,  3.38134766e-02,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -1.05773926e-01, -3.74755859e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           9.75036621e

In [55]:
angulo = 45
a_rad = np.deg2rad(angulo)
(1 - np.sin(a_rad))/1

0.29289321881345254

In [64]:
from rasterio.windows import Window

In [65]:
Window.from_slices((0,30000), (0, 1024))

Window(col_off=0, row_off=0, width=1024, height=30000)

In [ ]:
for k, m in enumerate(mdss):
    transform = transformations[k]
    profile.update(
        transform=transform,
        width=m.shape[0],
        height=m.shape[1]
    )

    with rasterio.open(f'../tmp/raw-{k}-{m.shape[1]}.tif', 'w', **profile) as dst:
        dst.write(m, 1)